In [29]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

In [34]:
df = pd.read_csv('input/autotel_with_dest_only.csv',index_col=0)

c:\projects\python\ssd\venv\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [35]:
df[:2]

,Car id,Order number,Category,Avg distance,Billing minutes,Address,Time,kmh,Coords,Lat,Lon,neighborhood,Area,Population,Street,City,Country,Street_c,Address_c,Address_c2,Date,Year,Month,Day,Hour,Minute,Temprature,Max Temprature,Min Temprature,Relative Humidity,Amount of Rain,Wind Speed,Wind Direction,Max Wind Speed per Min,Max Win Speed per 10 Min
0,46-525-81,60386,שריון חינמי,8,13,"הרב רובינשטיין 61, תל אביב יפו, ישראל",2017-10-23 00:31:00,37.15,"(32.0357011, 34.7581719)",32.04,34.76,(יפו ד' (גבעת התמרים,389591.69,6631.0,הרב רובינשטיין 61,תל אביב יפו,ישראל,הרב רובינשטיין,"הרב רובינשטיין 61, תל אביב יפו, ישראל","הרב רובינשטיין , תל אביב יפו, ישראל",2017-10-23 00:00:00,2017.0,10.0,23.0,0.0,31.0,20.53,20.8,20.1,66.67,0.0,2.53,117.0,3.2,2.7
1,48-437-81,60387,שריון חינמי,3,8,"ז'בוטינסקי 10, תל אביב יפו, ישראל",2017-10-23 00:21:00,21.56,"(32.0895831, 34.7734306)",32.09,34.77,הצפון הישן - החלק הצפוני,1819996.41,32080.0,ז'בוטינסקי 10,תל אביב יפו,ישראל,ז'בוטינסקי,"ז'בוטינסקי 10, תל אביב יפו, ישראל","ז'בוטינסקי , תל אביב יפו, ישראל",2017-10-23 00:00:00,2017.0,10.0,23.0,0.0,21.0,20.53,20.8,20.1,66.67,0.0,2.53,117.0,3.2,2.7


In [3]:
df['start_dt'] = pd.to_datetime(df['start_dt'], errors='coerce')
df['end_dt'] = pd.to_datetime(df['end_dt'],errors='coerce')

In [4]:
def hour_range_list(end_dt,next_start_dt,start_dt,dest_nei_id, origin_nei_id):
    l = pd.period_range(end_dt, next_start_dt, freq='H').values.tolist()
    return l

In [5]:
def diff_stay(dest_nei_id, origin_nei_id, end_dt, next_start_dt, start_dt):
    if origin_nei_id == dest_nei_id and end_dt.hour == start_dt.hour:
        return []
    else:
        return [end_dt]
    
    if origin_nei_id == dest_nei_id and end_dt.hour != start_dt.hour:
        return [end_dt]
    elif origin_nei_id != dest_nei_id and end_dt.hour != start_dt.hour:
        return [end_dt]
    elif origin_nei_id != dest_nei_id and end_dt.hour == start_dt.hour:
        return [end_dt]

In [6]:
gb = df.groupby('vehicle_id')
groups_vehicle_id = [gb.get_group(x).sort_values(by=['start_dt']) for x in gb.groups]

In [7]:
l_explode = []
for g in groups_vehicle_id:
    g_shift = g.shift(periods=-1).sort_values(by=['start_dt'])
    g["next_origin_nei_id"] = g_shift["origin_nei_id"]
    g["next_start_dt"] = g_shift["start_dt"]
    g['hour_list'] = g.apply(lambda x: hour_range_list(x.end_dt, x.next_start_dt, x.start_dt,x.dest_nei_id, x.origin_nei_id) if x.dest_nei_id == x.next_origin_nei_id else diff_stay(x.dest_nei_id, x.origin_nei_id, x.end_dt, x.next_start_dt, x.start_dt), axis=1)
    l_explode.append(g.explode('hour_list'))

In [8]:
r = pd.concat(l_explode)

In [9]:
r.head()

,vehicle_id,order_id,shiryun_cat,avg_dist,billing_minutes,orig_addr,dest_addr,start_dt,end_dt,kmh,orig_addr_c,dest_addr_c,orig_coord,dest_coord,orig_x,orig_y,dest_x,dest_y,orig_x_merc,orig_y_merc,dest_x_merc,dest_y_merc,start_hour,start_dayofweek,start_month,orig_nei,dest_nei,origin_nei_id,orig_nei_name,orig_nei_area,orig_nei_popu,dest_nei_id,dest_nei_name,dest_nei_area,dest_nei_popu,next_origin_nei_id,next_start_dt,hour_list
54658,156-01-201,100516,שריון חינמי,4,11.97,"טשרניחובסקי 39-47, תל אביב יפו, ישראל","הירקון 198, תל אביב יפו, ישראל",2017-12-19 18:33:47,2017-12-19 18:52:16.410,20.050125,"טשרניחובסקי 39, תל אביב יפו, ישראל","הירקון 198, תל אביב יפו, ישראל","(32.074096, 34.772972)","(32.0882654, 34.7716124)",32.074096,34.772972,32.088265,34.771612,3.870910e+06,3.773041e+06,3.870758e+06,3.774902e+06,18,Tuesday,December,לב תל-אביב,הצפון הישן - החלק הצפוני,37.0,לב תל-אביב,1977207.466,33239,30.0,הצפון הישן - החלק הצפוני,1819996.406,32080,30.0,2017-12-20 08:25:58,2017-12-19 18:00
54658,156-01-201,100516,שריון חינמי,4,11.97,"טשרניחובסקי 39-47, תל אביב יפו, ישראל","הירקון 198, תל אביב יפו, ישראל",2017-12-19 18:33:47,2017-12-19 18:52:16.410,20.050125,"טשרניחובסקי 39, תל אביב יפו, ישראל","הירקון 198, תל אביב יפו, ישראל","(32.074096, 34.772972)","(32.0882654, 34.7716124)",32.074096,34.772972,32.088265,34.771612,3.870910e+06,3.773041e+06,3.870758e+06,3.774902e+06,18,Tuesday,December,לב תל-אביב,הצפון הישן - החלק הצפוני,37.0,לב תל-אביב,1977207.466,33239,30.0,הצפון הישן - החלק הצפוני,1819996.406,32080,30.0,2017-12-20 08:25:58,2017-12-19 19:00
54658,156-01-201,100516,שריון חינמי,4,11.97,"טשרניחובסקי 39-47, תל אביב יפו, ישראל","הירקון 198, תל אביב יפו, ישראל",2017-12-19 18:33:47,2017-12-19 18:52:16.410,20.050125,"טשרניחובסקי 39, תל אביב יפו, ישראל","הירקון 198, תל אביב יפו, ישראל","(32.074096, 34.772972)","(32.0882654, 34.7716124)",32.074096,34.772972,32.088265,34.771612,3.870910e+06,3.773041e+06,3.870758e+06,3.774902e+06,18,Tuesday,December,לב תל-אביב,הצפון הישן - החלק הצפוני,37.0,לב תל-אביב,1977207.466,33239,30.0,הצפון הישן - החלק הצפוני,1819996.406,32080,30.0,2017-12-20 08:25:58,2017-12-19 20:00
54658,156-01-201,100516,שריון חינמי,4,11.97,"טשרניחובסקי 39-47, תל אביב יפו, ישראל","הירקון 198, תל אביב יפו, ישראל",2017-12-19 18:33:47,2017-12-19 18:52:16.410,20.050125,"טשרניחובסקי 39, תל אביב יפו, ישראל","הירקון 198, תל אביב יפו, ישראל","(32.074096, 34.772972)","(32.0882654, 34.7716124)",32.074096,34.772972,32.088265,34.771612,3.870910e+06,3.773041e+06,3.870758e+06,3.774902e+06,18,Tuesday,December,לב תל-אביב,הצפון הישן - החלק הצפוני,37.0,לב תל-אביב,1977207.466,33239,30.0,הצפון הישן - החלק הצפוני,1819996.406,32080,30.0,2017-12-20 08:25:58,2017-12-19 21:00
54658,156-01-201,100516,שריון חינמי,4,11.97,"טשרניחובסקי 39-47, תל אביב יפו, ישראל","הירקון 198, תל אביב יפו, ישראל",2017-12-19 18:33:47,2017-12-19 18:52:16.410,20.050125,"טשרניחובסקי 39, תל אביב יפו, ישראל","הירקון 198, תל אביב יפו, ישראל","(32.074096, 34.772972)","(32.0882654, 34.7716124)",32.074096,34.772972,32.088265,34.771612,3.870910e+06,3.773041e+06,3.870758e+06,3.774902e+06,18,Tuesday,December,לב תל-אביב,הצפון הישן - החלק הצפוני,37.0,לב תל-אביב,1977207.466,33239,30.0,הצפון הישן - החלק הצפוני,1819996.406,32080,30.0,2017-12-20 08:25:58,2017-12-19 22:00


In [10]:
# r['hour_list'] = pd.to_datetime(r['hour_list'],errors='coerce')
# r.groupby(['vehicle_id','dest_nei_id','hour_list']).count().reset_index()
# need to take only unique by vehicle_id and next_start_dt

In [11]:
r = r[r['hour_list'].notna()]

In [12]:
r.shape

(877705, 38)

In [13]:
r.drop_duplicates(subset=['vehicle_id','next_start_dt','hour_list'],inplace=True)

In [14]:
r["end_dt"] = pd.to_datetime(r["end_dt"])

In [15]:
r["Date"] = r["end_dt"].dt.date
r["Hour"] = r["end_dt"].dt.hour


In [16]:
tmp = r[['Date','Hour','dest_nei_name','hour_list']]

In [17]:
count = tmp.groupby(['Date','Hour','dest_nei_name']).count().reset_index()

In [18]:
count

,Date,Hour,dest_nei_name,hour_list
0,2017-10-23,0,(יפו ד' (גבעת התמרים,1
1,2017-10-23,0,בבלי,9
2,2017-10-23,0,הצפון הישן - החלק הצפוני,1
3,2017-10-23,0,הצפון הישן-החלק הדרומי,9
4,2017-10-23,0,התקוה,89
...,...,...,...,...
75819,2018-05-01,0,נוה צדק,1
75820,2018-05-01,0,עג'מי וגבעת עליה,1
75821,2018-05-01,0,פלורנטין,2
75822,2018-05-01,0,צפון יפו,1


In [19]:
count.rename(columns={'hour_list':'count','dest_nei_name':'neighborhood'},inplace=True)

In [20]:
count.to_csv("./input/targets.csv", encoding='utf-8-sig')